# Setting up your own custom task

If the available task methodologies are not suitable for your use case, this guide explains how to set up your own task from scratch.
Using the task interface gives us the added benefit of getting built-in input and output logging and gives us the possibility of using the evaluation framework.

For the purpose of this tutorial, we will set up a simple keyword extraction task.
To do so, we will leverage `luminous-base` and a few-shot prompt to generate matching keywords for variable input texts.
Next, we will build an evaluator to check how well our extractor performs.

Let's start with the interface of any generic task. The full `Task` interface can be found here: [task.py](task.py).
However, to initially set up a `Task`, there are only a few parts relevant to us. For now, we shall only care about the following part of the interface:

```python
Input = TypeVar("Input", bound=PydanticSerializable)
Output = TypeVar("Output", bound=PydanticSerializable)

class Task(ABC, Generic[Input, Output]):
    @abstractmethod
    def run(self, input: Input, logger: DebugLogger) -> Output:
        """Executes the process for this use-case."""
        ...
```

For every task, we have to define an `Input`, an `Output` and how we would like to run it. Since these can vary so much, we make no assumptions about a `Task`'s implementation. 
We only require both input and output to be `PydanticSerializable`. The best way to guarantee this is to make them pydantic `BaseModel`s. For our keyword extraction task, we could define `Input` and `Output` as follows:

In [11]:
from pydantic import BaseModel

class KeywordExtractionInput(BaseModel):
    """This is the text we will extract keywords from"""
    text: str

class KeywordExtractionOutput(BaseModel):
    """The matching set of keywords we aim to extract"""
    keywords: set[str]

Now that we have our input and output defined, we implement the actual task.

The steps that the task consists of are:
- Create a `Prompt` using the input text
- Have `luminous-base` complete the prompt
- Extract keywords from said completion

When a task is executed, we offer the possibility to log all intermediate steps and outputs.
This is crucial because large language models are inherently probabilistic.
Therefore, we might get unexpected answers.
This logging allows us to check the results afterwards and find out what went wrong.

For this, we shall inject an `InMemoryDebugLogger` into the task. For more information, check the logger's implementation in [task.py](task.py).

In [12]:
from aleph_alpha_client import Client, CompletionRequest, Prompt
from intelligence_layer.completion import Completion, CompletionInput
from intelligence_layer.task import DebugLogger, Task


class KeywordExtractionTask(Task[KeywordExtractionInput, KeywordExtractionOutput]):
    PROMPT_TEMPLATE: str = """Identify matching keywords for each text.
###
Text: The "Whiskey War" is an ongoing conflict between Denmark and Canada over ownership of Hans Island. The dispute began in 1973, when Denmark and Canada reached an agreement on Greenland's borders. However, no settlement regarding Hans Island could be reached by the time the treaty was signed. Since then both countries have used peaceful means - such as planting their national flag or burying liquor - to draw attention to the disagreement.
Keywords: Conflict, Whiskey War, Denmark, Canada, Treaty, Flag, Liquor
###
Text: I really like pizza and sushi.
Keywords: Pizza, Sushi
###
Text: NASA launched the Discovery program to explore the solar system. It comprises a series of expeditions that have continued from the program's launch in the 1990s to the present day. In the course of the 16 expeditions launched so far, the Moon, Mars, Mercury and Venus, among others, have been explored. Unlike other space programs, the Discovery program places particular emphasis on cost efficiency, true to the motto: "faster, better, cheaper".
Keywords: Space program, NASA, Expedition, Cost efficiency, Moon, Mars, Mercury, Venus
###
Text: {text}
Keywords:"""
    MODEL: str = "luminous-base"

    def __init__(self, client: Client) -> None:
        super().__init__()
        self._client = client
        self._completion_task = Completion(client)

    def _create_prompt(self, text: str) -> CompletionRequest:
        prompt = Prompt.from_text(self.PROMPT_TEMPLATE.format(text=text))
        # Explain stop sequences here.
        request = CompletionRequest(
            prompt=prompt,
            stop_sequences=["\n", "###"],
            frequency_penalty=0.25
        )
        return request

    def run(self, input: KeywordExtractionInput, logger: DebugLogger) -> KeywordExtractionOutput:
        completion_input = CompletionInput(request=self._create_prompt(input.text), model=self.MODEL)
        completion = self._completion_task.run(
            completion_input, logger
        )
        return KeywordExtractionOutput(keywords=set(k.strip().lower() for k in completion.completion().split(",") if k.strip()))


Now, we can run this `KeywordExtractionTask` like so:

In [13]:
from os import getenv

from intelligence_layer.task import InMemoryDebugLogger


client = Client(getenv("AA_TOKEN"))
task = KeywordExtractionTask(client)
text = "Computer vision describes the processing of an image by a machine using external devices (e.g., a scanner) into a digital description of that image for further processing. An example of this is optical character recognition (OCR), the recognition and processing of images containing text. Further processing and final classification of the image is often done using artificial intelligence methods. The goal of this field is to enable computers to process visual tasks that were previously reserved for humans."

input = KeywordExtractionInput(text=text)
logger = InMemoryDebugLogger(name="Keyword Extraction")
output = task.run(input, logger)

print(output)

keywords={'artificial intelligence', 'optical character recognition', 'image processing', 'computer vision'}


Looks great!

Now that our task is set up, we can start evaluating its performance.

For this, we will have to set up an evaluator. The full interface for this `Evaluator` can be found [here](task.py).

```python
class Evaluator(ABC, Generic[Input, ExpectedOutput, Evaluation, AggregatedEvaluation]):
    @abstractmethod
    def evaluate(
        self,
        input: Input,
        logger: DebugLogger,
        expected_output: ExpectedOutput,
    ) -> Evaluation:
        """Executes the evaluation for this use-case."""
        pass
```

Notice that, just like our `Task`, this `Evaluator` takes an `Input`. This input is, in fact, identical to our task input.
However, we don't just want to run a task, we also want to evaluate the result. Therefore, our evaluator also depends on some `ExpectedOutput`, as well `Evaluation`.
We will come back to `AggregatedEvaluation` at a later stage.

Let's build an evaluator that can check the performance of our keyword extraction methodology. For this, we need four things:
- An implementation of the task to be run (we suggest supplying this in the `Evaluator`'s `__init__`)
- An interface for our `ExpectedOutput`
- Some `Evaluation`, i.e., the output of the `evaluate` method
- An implementation of the `evaluate` function.

In our case, we will measure the performance of our keyword extraction by calculating the proportion of correctly generated keywords compared to all expected keywords. 
This is also known as the "true positive rate". 
To calculate this, our evaluate function will need a set of the expected keywords.


In [14]:
class KeywordExtractionExpectedOutput(BaseModel):
    """This is the expected output for an example run. This is used to compare the output of the task with.

    We will be evaluating our keyword extraction based on the expected keywords. """
    keywords: set[str]

class KeywordExtractionEvaluation(BaseModel):
    """This is the interface for the metrics that are generated for each evaluation case"""
    true_positive_rate: float 
    true_positives: set[str]
    false_positives: set[str]
    false_negatives: set[str]

Accordingly, our evaluate function will take a `KeywordExtractionInput`, and run the task with this.
Next, we shall compare the generated output with the `KeywordExtractionExpectedOutput` to create the `KeywordExtractionEvaluation`.

```python
def evaluate(
        self,
        input: KeywordExtractionInput,
        logger: DebugLogger,
        expected_output: KeywordExtractionExpectedOutput,
    ) -> KeywordExtractionEvaluation:
        output = self.task.run(input, logger)
        true_positives = expected_output.keywords & output.keywords
        false_positives = output.keywords - true_positives
        false_negatives = true_positives - output.keywords
        return KeywordExtractionEvaluation(true_positive_rate=len(true_positives) / len(expected_output.keywords), 
                                           true_positives=true_positives, 
                                           false_positive=false_positives, 
                                           false_negatives=false_negatives)
```

However, to quantitatively evaluate the performance of a task, we will need to run many different examples and calculate the metrics for each. 
To do this, we can use the `evaluate_dataset` function provided by the `Evaluator` base class. This takes a dataset, runs all the examples, and aggregates the metrics generated from the evaluation.

To set this up, we will first need to create an interface for the `AggregatedEvaluation` and implement the `aggregate` method.

In [15]:
"""This is the interface for the aggregated metrics that are generated from running a number of examples"""
class KeywordExtractionAggregatedEvaluation(BaseModel):
    average_true_positive_rate: float

Now that we have all parts in place, let's implement our `KeywordExtractionEvaluator`.

In [16]:
from statistics import mean
from typing import Sequence
from intelligence_layer.task import Evaluator

class KeywordExtractionEvaluator(Evaluator[KeywordExtractionInput, KeywordExtractionExpectedOutput, KeywordExtractionEvaluation, KeywordExtractionAggregatedEvaluation]):
    def __init__(self, task: KeywordExtractionTask) -> None:
        """We recommend adding the task to the init method of the evaluator
        
        This allows for easy comparing of different implementations of the same task."""
        self.task = task


    def evaluate(
        self,
        input: KeywordExtractionInput,
        logger: DebugLogger,
        expected_output: KeywordExtractionExpectedOutput,
    ) -> KeywordExtractionEvaluation:
        output = self.task.run(input, logger.child_logger("Keyword Extraction"))
        true_positives = output.keywords & expected_output.keywords
        false_positives = output.keywords - expected_output.keywords 
        false_negatives = expected_output.keywords - output.keywords
        evaluation = KeywordExtractionEvaluation(true_positive_rate=len(true_positives) / len(expected_output.keywords), 
                                           true_positives=true_positives, 
                                           false_positives=false_positives, 
                                           false_negatives=false_negatives)
        eval_logger = logger.child_logger("Evaluation results")
        eval_logger.log("Metrics", evaluation)
        return evaluation
         

    def aggregate(self, evaluations: Sequence[KeywordExtractionEvaluation]) -> KeywordExtractionAggregatedEvaluation:
        true_positive_rate = mean(e.true_positive_rate for e in evaluations)
        return KeywordExtractionAggregatedEvaluation(average_true_positive_rate=true_positive_rate)

Let's run this on a single example.

In [17]:
evaluator = KeywordExtractionEvaluator(task=task)

logger = InMemoryDebugLogger(name="Evaluation logger")
input = KeywordExtractionInput(text="This is a text about dolphins and sharks.")
expected_output = KeywordExtractionExpectedOutput(keywords=["dolphins", "sharks"])
evaluation = evaluator.evaluate(input, 
                                logger, 
                                expected_output)
print(evaluation)

true_positive_rate=1.0 true_positives={'dolphins', 'sharks'} false_positives=set() false_negatives=set()


Now that we have implemented our aggregate method, let's run a dataset with some example data.

In [18]:
from intelligence_layer.task import Dataset, Example

dataset = Dataset(
    name="Keyword extraction dataset",
    examples=[
        Example(
            input=input,
            expected_output=expected_output
        ), 
        Example(
            input=KeywordExtractionInput(
                text="Clinical psychology is an integration of human science, behavioral science, theory, and clinical knowledge for the purpose of understanding, preventing, and relieving psychologically-based distress or dysfunction and to promote subjective well-being and personal development."
            ),
            expected_output=KeywordExtractionExpectedOutput(
                keywords={"clinical psychology", "well-being", "personal development"}
            )
        ),
        Example(
            input=KeywordExtractionInput(
                text="Prospect theory is a theory of behavioral economics, judgment and decision making that was developed by Daniel Kahneman and Amos Tversky in 1979.[1] The theory was cited in the decision to award Kahneman the 2002 Nobel Memorial Prize in Economics.[2]Based on results from controlled studies, it describes how individuals assess their loss and gain perspectives in an asymmetric manner (see loss aversion)."
            ),
            expected_output=KeywordExtractionExpectedOutput(
                keywords={"prospect theory", "behavioural economics", "decision making", "losses and gains"}
            )
        )
    ]
)
logger = InMemoryDebugLogger(name="Evaluate dataset debug logger")

aggregated_evaluations = evaluator.evaluate_dataset(dataset, logger)
print(aggregated_evaluations)

Evaluating: 100%|██████████| 3/3 [00:01<00:00,  1.82it/s]

average_true_positive_rate=0.6388888888888888


We have now run our first evaluation on this tiny dataset.
Let's have a more detailed look at the performance by inspecting the debug log.

In [19]:
logger.logs[-1] # this will give us the debug log of the last example that was run

2
├── Keyword Extraction
│   ├── ╭───────────────────────────────────────────────── Input ─────────────────────────────────────────────────╮
│   │   │ {                                                                                                       │
│   │   │   "text": "Prospect theory is a theory of behavioral economics, judgment and decision making that was   │
│   │   │ developed by Daniel Kahneman and Amos Tversky in 1979.[1] The theory was cited in the decision to award │
│   │   │ Kahneman the 2002 Nobel Memorial Prize in Economics.[2]Based on results from controlled studies, it     │
│   │   │ describes how individuals assess their loss and gain perspectives in an asymmetric manner (see loss     │
│   │   │ aversion)."                                                                                             │
│   │   │ }                                                                                                       │
│   │   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
│   ├── ╭───────────────────────────────────────────────── Input ─────────────────────────────────────────────────╮
│   │   │ {                                                                                                       │
│   │   │   "request": [                                                                                          │
│   │   │     {                                                                                                   │
│   │   │       "items": [                                                                                        │
│   │   │         [                                                                                               │
│   │   │           "Identify matching keywords for each text.\n###\nText: The \"Whiskey War\" is an ongoing      │
│   │   │ conflict between Denmark and Canada over ownership of Hans Island. The dispute began in 1973, when      │
│   │   │ Denmark and Canada reached an agreement on Greenland's borders. However, no settlement regarding Hans   │
│   │   │ Island could be reached by the time the treaty was signed. Since then both countries have used peaceful │
│   │   │ means - such as planting their national flag or burying liquor - to draw attention to the               │
│   │   │ disagreement.\nKeywords: Conflict, Whiskey War, Denmark, Canada, Treaty, Flag, Liquor\n###\nText: I     │
│   │   │ really like pizza and sushi.\nKeywords: Pizza, Sushi\n###\nText: NASA launched the Discovery program to │
│   │   │ explore the solar system. It comprises a series of expeditions that have continued from the program's   │
│   │   │ launch in the 1990s to the present day. In the course of the 16 expeditions launched so far, the Moon,  │
│   │   │ Mars, Mercury and Venus, among others, have been explored. Unlike other space programs, the Discovery   │
│   │   │ program places particular emphasis on cost efficiency, true to the motto: \"faster, better,             │
│   │   │ cheaper\".\nKeywords: Space program, NASA, Expedition, Cost efficiency, Moon, Mars, Mercury,            │
│   │   │ Venus\n###\nText: Prospect theory is a theory of behavioral economics, judgment and decision making     │
│   │   │ that was developed by Daniel Kahneman and Amos Tversky in 1979.[1] The theory was cited in the decision │
│   │   │ to award Kahneman the 2002 Nobel Memorial Prize in Economics.[2]Based on results from controlled        │
│   │   │ studies, it describes how individuals assess their loss and gain perspectives in an asymmetric manner   │
│   │   │ (see loss aversion).\nKeywords:",                                                                       │
│   │   │           []                                                                                            │
│   │   │         ]                                                                                               │
│   │   │       ]              

Let's inspect this debug log from top to bottom to try and figure out what happened here.

1. **Input**: This corresponds to the `Input` we supplied for our task. In this case, it's just the text of the provided example.

2. **Prompt template**: The prompt template used by our task. Note the placeholder for `text`.

3. **Original request & response**: The request and response objects supplied to and from the Aleph Alpha API.
Two things are of particular interest here.
- The formatted prompt (here, we can check whether anything went wrong during formatting)
- The original completion created by the API.

4. **Output**: The output of our task. Here this is just a list of stripped, lowercase keywords.

5. **Metrics**: Several metrics generated by our `KeywordExtractionTaskEvaluator`.

Let's have a look at the evaluation results.
Here, we can see that the model returned "behavi*o*ral economics" as a keyword.
However, in the `false_negatives`, we can see that we did indeed expect this phrase, but with a different spelling: "behavi*ou*ral economics".
Thus, the debug log helped us easily identify this misalignment between our dataset and the model's generation.


**What does this tell us?**

Why did the British "ou" and the American "o" go to therapy?

They had behavioural differences!